In [1]:
import os
import sys
import random
from PIL import Image
import numpy as np
import pandas as pd
#for image transform
import cv2

import matplotlib
import matplotlib.pyplot as plt
matplotlib.rcParams['figure.figsize'] = [5, 5]
matplotlib.rcParams['figure.dpi'] = 200

import random
import time

sys.path.insert(1, '//workspace/dl2020')
#from model_loader_CP2 import *
from CP_helper_RCNN import *
from data_helper_RCNN import *

#from Unet import *

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import torch.utils.data


 
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor
#from engine import train_one_epoch, evaluate


from data_helper import UnlabeledDataset, LabeledDataset
from helper import collate_fn, draw_box


In [2]:
random.seed(0)
np.random.seed(0)
torch.manual_seed(0);
image_folder = '../data'
annotation_csv = '../data/annotation.csv'

In [3]:
# import torchvision
# from torchvision.models.detection import FasterRCNN
# from torchvision.models.detection.rpn import AnchorGenerator
# from torchvision.models.detection.backbone_utils import resnet_fpn_backbone

# # load a pre-trained model for classification and return
# # only the features
# #backbone = torchvision.models.mobilenet_v2(pretrained=True).features
# backbone = resnet_fpn_backbone('resnet18', pretrained = False)
# # FasterRCNN needs to know the number of
# # output channels in a backbone. For mobilenet_v2, it's 1280
# # so we need to add it here
# #backbone.out_channels = 1280
# backbone.out_channels = 256 #this is a guess


# # let's make the RPN generate 5 x 3 anchors per spatial
# # location, with 5 different sizes and 3 different aspect
# # ratios. We have a Tuple[Tuple[int]] because each feature
# # map could potentially have different sizes and
# # aspect ratios
# anchor_generator = AnchorGenerator(sizes=((32, 64, 128, 256, 512),),
#                                    aspect_ratios=((0.5, 1.0, 2.0),))

# # let's define what are the feature maps that we will
# # use to perform the region of interest cropping, as well as
# # the size of the crop after rescaling.
# # if your backbone returns a Tensor, featmap_names is expected to
# # be [0]. More generally, the backbone should return an
# # OrderedDict[Tensor], and in featmap_names you can choose which
# # feature maps to use.
# roi_pooler = torchvision.ops.MultiScaleRoIAlign(featmap_names=[0],
#                                                 output_size=7,
#                                                 sampling_ratio=2)

# # put the pieces together inside a FasterRCNN model
# model = FasterRCNN(backbone,
#                    num_classes=2,
#                    rpn_anchor_generator=anchor_generator,
#                    box_roi_pool=roi_pooler)

In [4]:
unlabeled_scene_index = np.arange(106)
# The scenes from 106 - 133 are labeled
# You should devide the labeled_scene_index into two subsets (training and validation)
labeled_scene_index = np.arange(106, 134)
train_labeled_scene_index, val_labeled_scene_index  = gen_train_val_index(labeled_scene_index)

In [5]:
kwargs = {
    #'first_dim': 'sample',
    'transform': transforms.ToTensor(),
    'image_folder': image_folder,
    'annotation_file': annotation_csv,
    'extra_info': True}

#dataset_train = LabeledDataset_RCNN (scene_index=train_labeled_scene_index, **kwargs)
#dataset_val = LabeledDataset_RCNN (scene_index=val_labeled_scene_index, **kwargs)
 
dataset_train = LabeledDataset(scene_index=train_labeled_scene_index, **kwargs)
dataset_val = LabeledDataset(scene_index=val_labeled_scene_index, **kwargs)

train_data_loader = torch.utils.data.DataLoader(
    dataset_train, batch_size=1, shuffle=False, num_workers=4,
    collate_fn=collate_fn)

val_data_loader = torch.utils.data.DataLoader(
    dataset_val, batch_size=1, shuffle=False, num_workers=4,
    collate_fn=collate_fn)

In [6]:
#mod = torchvision.models.resnet50()

In [7]:
for i, (sample, old_targets, road_image, extra) in enumerate(val_data_loader): #, print_freq, header): 
        
        images = sample
        targets = trans_target(old_targets)
        #print("images len {}, targets len {}".format(len(images), len(targets)))
        #print("images[0] shape {}".format(images[0].shape)) # [6, 3, 256, 306]      
        #images = list(image.to(device) for image in images)
        #targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
        break

In [8]:
# sample2 = [s.sum(dim=0).squeeze(0).to(device) for s in sample]
# targets2 = [{k: v.to(device) for k, v in t.items()} for t in targets]

In [26]:
from torchvision.models.detection.rpn import AnchorGenerator
from torchvision.models.detection.rpn import RPNHead
def fr50_Model(pretrained = False):
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=pretrained) #true works
    # create an anchor_generator for the FPN
    # which by default has 5 outputs
    anchor_generator = AnchorGenerator(
        #sizes=tuple([(16, 32, 64, 128, 256, 512) for _ in range(5)]),
        sizes=tuple([(10, 15, 20, 30, 40) for _ in range(5)]),
         
        aspect_ratios=tuple([(0.25, 0.5, 1.0, 2.0) for _ in range(5)]))
    
    
    
    model.rpn.anchor_generator = anchor_generator

    # 256 because that's the number of features that FPN returns
    model.rpn.head = RPNHead(256, anchor_generator.num_anchors_per_location()[0])
    return model

In [10]:
# model = model.to(device)

# fastRCNNout = model(sample2, targets2)

In [11]:
# fastRCNNout

In [13]:
# sample3 = [s.sum(dim=0).squeeze(0).to(device) for s in sample]
# targets3 = [{k: v.to(device) for k, v in t.items()} for t in targets]

In [14]:
# model.eval()
# with torch.no_grad():
#      prediction = model(sample3, None)

In [15]:
# prediction #no pretrained it doens't predict anything, but that' sfine
# [{'boxes': tensor([], device='cuda:0', size=(0, 4)),
#   'labels': tensor([], device='cuda:0', dtype=torch.int64),
#   'scores': tensor([], device='cuda:0')}]

In [16]:
sample[0].shape

torch.Size([6, 3, 256, 306])

In [17]:
#tt = transforms.Compose([transforms.Resize((800, 800)), transforms.ToTensor()])

In [18]:
#sample3_sew = [tt(sew_images(s)).to(device) for s in sample]

In [19]:
# model.eval()
# with torch.no_grad():
#      prediction_sew = model(sample3_sew, None)

In [20]:
#prediction_sew

In [27]:
## try training the a model 

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# our dataset has two classes only - background and person
num_classes = 2 #4 returned classifier as nan

# get the model using our helper function
#model = get_instance_segmentation_model(num_classes)
model = fr50_Model(True) #try true first

# move model to the right device
model.to(device)

# construct an optimizer
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.0001, #reduce from 0.005 to help with the classifer loss = nan issue
                            momentum=0.9, weight_decay=0.0005)

# and a learning rate scheduler which decreases the learning rate by
# 10x every 3 epochs
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                               step_size=3,
                                               gamma=0.1)

In [28]:
num_epochs = 1

for epoch in range(num_epochs):
    # train for one epoch, printing every 10 iterations
    train_one_epoch_FastRCNN(model, optimizer, train_data_loader, device, epoch, print_freq=100)
    # update the learning rate
    lr_scheduler.step()
    # evaluate on the test dataset - this has issues
    #evaluate(model, val_data_loader, device=device)

Epoch: [0]  [   0/2394]  eta: 0:52:42  lr: 0.000100  loss: 69.2373 (69.2373)  loss_classifier: 0.0570 (0.0570)  loss_box_reg: 0.0006 (0.0006)  loss_objectness: 0.6901 (0.6901)  loss_rpn_box_reg: 68.4895 (68.4895)  time: 1.3211  data: 0.5603  max mem: 5066
Epoch: [0]  [ 100/2394]  eta: 0:14:52  lr: 0.000100  loss: 73.1177 (76.3359)  loss_classifier: 0.1342 (0.1124)  loss_box_reg: 0.0015 (0.0012)  loss_objectness: 0.6599 (0.6769)  loss_rpn_box_reg: 72.3232 (75.5454)  time: 0.3794  data: 0.0071  max mem: 5066
Epoch: [0]  [ 200/2394]  eta: 0:13:50  lr: 0.000100  loss: 44.7692 (72.0881)  loss_classifier: 0.1122 (0.0954)  loss_box_reg: 0.0012 (0.0010)  loss_objectness: 0.0029 (0.5230)  loss_rpn_box_reg: 44.6627 (71.4687)  time: 0.3553  data: 0.0073  max mem: 5066
Epoch: [0]  [ 300/2394]  eta: 0:13:00  lr: 0.000100  loss: 28.2854 (60.5151)  loss_classifier: 0.0487 (0.0932)  loss_box_reg: 0.0005 (0.0010)  loss_objectness: 0.0019 (0.3504)  loss_rpn_box_reg: 28.2348 (60.0705)  time: 0.3668  data

KeyboardInterrupt: 

In [29]:
#give me another one
tt = transforms.Compose([transforms.Resize((800, 800)), transforms.ToTensor()]) #this is for 6 images combo
for i, (sample, old_targets, road_image, extra) in enumerate(val_data_loader): #, print_freq, header): 
        
        images = [tt(sew_images(s)).to(device) for s in sample] #list of [3, 800, 800], should be 1 per patch
        targets = trans_target(old_targets)
        
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
        break

In [30]:
model.eval()
with torch.no_grad():
     prediction = model(images)

In [31]:
prediction

[{'boxes': tensor([], device='cuda:0', size=(0, 4)),
  'labels': tensor([], device='cuda:0', dtype=torch.int64),
  'scores': tensor([], device='cuda:0')}]

In [25]:
targets

[{'boxes': tensor([[ 587.7640, -410.7347,  634.5888, -390.4039],
          [ 723.5701, -515.9766,  770.2949, -494.8876],
          [ 346.5826, -643.8907,  392.2856, -625.2436],
          [ 208.3556, -448.8516,  252.5559, -429.6745],
          [ 103.3553, -449.4175,  147.5497, -430.6902],
          [  31.1752, -483.2730,   79.2614, -463.0602],
          [ 340.5685, -484.8769,  384.6458, -465.9312],
          [ 648.6622, -573.7671,  696.0050, -555.0985],
          [ 496.1005, -604.4378,  541.8034, -585.7907],
          [ 519.1313, -483.1805,  561.6338, -464.6352],
          [  25.9092, -382.5251,   71.8441, -359.7119],
          [ 716.3273, -636.7295,  763.7090, -617.2219],
          [ 321.8503, -448.2401,  395.7314, -425.9930],
          [ 271.5695, -608.9213,  317.2724, -590.2742],
          [ 384.5264, -608.4007,  430.2294, -589.7536],
          [  66.1751, -413.8457,  110.3829, -394.0988],
          [ 256.3764, -411.0828,  300.9099, -392.4210],
          [ 692.8306, -481.0863,  740.1